In [1]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"

In [3]:
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram
from qiskit_machine_learning.algorithms import VQC
from sklearn.preprocessing import LabelEncoder
from qiskit.circuit.library import PauliFeatureMap, ZFeatureMap, ZZFeatureMap
from qiskit_machine_learning.circuit.library import RawFeatureVector
from sklearn.preprocessing import MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.decomposition import PCA
from sklearn.datasets import load_wine
import matplotlib.pyplot as plt
import seaborn as sns


import numpy as np
import gym

# Carica il dataset Iris
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Caricamento dei dati
data = load_iris()
X = data['data']
y = data['target']

# Preprocessing
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2    , random_state=42, stratify=y)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
print(X_train)


# Creazione di un circuito iniziale

(120, 4) (120,) (30, 4) (30,)
[[0.02777778 0.375      0.06779661 0.04166667]
 [0.16666667 0.20833333 0.59322034 0.66666667]
 [0.69444444 0.33333333 0.6440678  0.54166667]
 [0.16666667 0.45833333 0.08474576 0.        ]
 [0.33333333 0.20833333 0.50847458 0.5       ]
 [0.55555556 0.20833333 0.6779661  0.75      ]
 [0.36111111 0.29166667 0.54237288 0.5       ]
 [0.55555556 0.33333333 0.69491525 0.58333333]
 [0.94444444 0.41666667 0.86440678 0.91666667]
 [0.94444444 0.75       0.96610169 0.875     ]
 [0.91666667 0.41666667 0.94915254 0.83333333]
 [0.47222222 0.375      0.59322034 0.58333333]
 [0.19444444 0.         0.42372881 0.375     ]
 [0.41666667 0.29166667 0.52542373 0.375     ]
 [0.41666667 0.25       0.50847458 0.45833333]
 [0.11111111 0.5        0.10169492 0.04166667]
 [0.22222222 0.75       0.08474576 0.08333333]
 [0.66666667 0.20833333 0.81355932 0.70833333]
 [0.58333333 0.45833333 0.76271186 0.70833333]
 [0.30555556 0.58333333 0.08474576 0.125     ]
 [0.55555556 0.20833333 0.6610

In [4]:
import pandas as pd
from sklearn.datasets import load_wine

# Carica il dataset Wine
wine_data = load_wine()
X = pd.DataFrame(wine_data.data, columns=wine_data.feature_names)
y = wine_data.target

# Aggiungi il target come colonna al DataFrame delle feature
X['target'] = y

# Calcola la correlazione tra le feature e il target
correlation_matrix = X.corr()

# Seleziona le correlazioni con il target
correlation_with_target = correlation_matrix['target'].drop('target')

# Ordina le correlazioni in ordine decrescente
correlation_with_target_sorted = correlation_with_target.sort_values(ascending=False)

# Stampa le correlazioni con il target
print("Correlazioni con il target:")
print(correlation_with_target_sorted)

# Identifica le colonne da droppare
# Seleziona le feature con correlazione compresa tra 0.5 e -0.4
columns_to_drop = ['malic_acid', 'color_intensity', 'ash','magnesium','alcohol','proanthocyanins','hue']

# Droppa le colonne dal DataFrame
df_dropped = X.drop(columns=columns_to_drop,inplace=True)

print("\nMatrice di correlazione:")
print(correlation_matrix)
print("\nDataFrame dopo il drop:")
print(df_dropped)


Correlazioni con il target:
alcalinity_of_ash               0.517859
nonflavanoid_phenols            0.489109
malic_acid                      0.437776
color_intensity                 0.265668
ash                            -0.049643
magnesium                      -0.209179
alcohol                        -0.328222
proanthocyanins                -0.499130
hue                            -0.617369
proline                        -0.633717
total_phenols                  -0.719163
od280/od315_of_diluted_wines   -0.788230
flavanoids                     -0.847498
Name: target, dtype: float64

Matrice di correlazione:
                               alcohol  malic_acid       ash  \
alcohol                       1.000000    0.094397  0.211545   
malic_acid                    0.094397    1.000000  0.164045   
ash                           0.211545    0.164045  1.000000   
alcalinity_of_ash            -0.310235    0.288500  0.443367   
magnesium                     0.270798   -0.054575  0.286587   


In [5]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)
# Verifica la divisione
print("Train set shape:", X_train.shape, y_train.shape)
print("Test set shape:", X_test.shape, y_test.shape)


Train set shape: (142, 7) (142,)
Test set shape: (36, 7) (36,)


In [6]:
# Caricamento dei dati
data = load_iris()
X = data['data']
y = data['target']

# Preprocessing
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2    , random_state=42, stratify=y)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
print(X_train)


(120, 4) (120,) (30, 4) (30,)
[[0.02777778 0.375      0.06779661 0.04166667]
 [0.16666667 0.20833333 0.59322034 0.66666667]
 [0.69444444 0.33333333 0.6440678  0.54166667]
 [0.16666667 0.45833333 0.08474576 0.        ]
 [0.33333333 0.20833333 0.50847458 0.5       ]
 [0.55555556 0.20833333 0.6779661  0.75      ]
 [0.36111111 0.29166667 0.54237288 0.5       ]
 [0.55555556 0.33333333 0.69491525 0.58333333]
 [0.94444444 0.41666667 0.86440678 0.91666667]
 [0.94444444 0.75       0.96610169 0.875     ]
 [0.91666667 0.41666667 0.94915254 0.83333333]
 [0.47222222 0.375      0.59322034 0.58333333]
 [0.19444444 0.         0.42372881 0.375     ]
 [0.41666667 0.29166667 0.52542373 0.375     ]
 [0.41666667 0.25       0.50847458 0.45833333]
 [0.11111111 0.5        0.10169492 0.04166667]
 [0.22222222 0.75       0.08474576 0.08333333]
 [0.66666667 0.20833333 0.81355932 0.70833333]
 [0.58333333 0.45833333 0.76271186 0.70833333]
 [0.30555556 0.58333333 0.08474576 0.125     ]
 [0.55555556 0.20833333 0.6610

In [7]:
import pandas as pd
from sklearn.datasets import load_wine

# Carica il dataset Wine
wine_data = load_wine()
X = wine_data.data
y = wine_data.target
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
# PCA di X
pca = PCA(n_components=4)
X = pca.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3   , random_state=42, stratify=y)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(124, 4) (124,) (54, 4) (54,)


In [8]:
num_features = X.shape[1]

feature_map = ZZFeatureMap(feature_dimension=num_features, reps=2, entanglement="linear")
feature_map.draw(output='mpl')
feature_map.decompose().draw(output='mpl')
print("Feature map dimension:", feature_map.num_parameters)
print(feature_map)


Feature map dimension: 4
     ┌────────────────────────────────────┐
q_0: ┤0                                   ├
     │                                    │
q_1: ┤1                                   ├
     │  ZZFeatureMap(x[0],x[1],x[2],x[3]) │
q_2: ┤2                                   ├
     │                                    │
q_3: ┤3                                   ├
     └────────────────────────────────────┘


In [8]:
"""import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

class QNetwork(nn.Module):
    def __init__(self, input_shape, num_actions):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(input_shape, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, num_actions)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

# Creazione del modello
input_shape = 4  # Numero di features dell'input (es. dimensioni dell'osservazione)
num_actions = 2  # Numero di azioni possibili

model = QNetwork(input_shape, num_actions)

# Ottimizzatore e perdita
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()
"""



"import torch\nimport torch.nn as nn\nimport torch.optim as optim\nimport torch.nn.functional as F\nimport numpy as np\n\nclass QNetwork(nn.Module):\n    def __init__(self, input_shape, num_actions):\n        super(QNetwork, self).__init__()\n        self.fc1 = nn.Linear(input_shape, 128)\n        self.fc2 = nn.Linear(128, 128)\n        self.fc3 = nn.Linear(128, num_actions)\n    \n    def forward(self, x):\n        x = F.relu(self.fc1(x))\n        x = F.relu(self.fc2(x))\n        return self.fc3(x)\n\n# Creazione del modello\ninput_shape = 4  # Numero di features dell'input (es. dimensioni dell'osservazione)\nnum_actions = 2  # Numero di azioni possibili\n\nmodel = QNetwork(input_shape, num_actions)\n\n# Ottimizzatore e perdita\noptimizer = optim.Adam(model.parameters(), lr=0.001)\nloss_fn = nn.MSELoss()\n"

In [9]:
"""class DQNAgent:
    def __init__(self, model, optimizer, loss_fn, num_actions, epsilon=1.0, epsilon_decay=0.995, epsilon_min=0.01, gamma=0.99):
        self.model = model
        self.optimizer = optimizer
        self.loss_fn = loss_fn
        self.num_actions = num_actions
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.gamma = gamma
        self.memory = []
        self.batch_size = 32
        self.max_memory_size = 1000
    
    def remember(self, state, action, reward, next_state, done):
        if len(self.memory) > self.max_memory_size:
            self.memory.pop(0)
        self.memory.append((state, action, reward, next_state, done))
    
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.num_actions)
        state = torch.FloatTensor(state).unsqueeze(0)
        q_values = self.model(state)
        return torch.argmax(q_values).item()
    
    def replay(self):
        if len(self.memory) < self.batch_size:
            return
        
        batch = np.random.choice(len(self.memory), self.batch_size, replace=False)
        for i in batch:
            state, action, reward, next_state, done = self.memory[i]
            
            state = torch.FloatTensor(state).unsqueeze(0)
            next_state = torch.FloatTensor(next_state).unsqueeze(0)
            target = reward
            
            if not done:
                target = reward + self.gamma * torch.max(self.model(next_state)).item()
            
            target_f = self.model(state)
            target_f[0][action] = target
            
            self.optimizer.zero_grad()
            output = self.model(state)
            loss = self.loss_fn(output, target_f)
            loss.backward()
            self.optimizer.step()
        
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            
"""
            


'class DQNAgent:\n    def __init__(self, model, optimizer, loss_fn, num_actions, epsilon=1.0, epsilon_decay=0.995, epsilon_min=0.01, gamma=0.99):\n        self.model = model\n        self.optimizer = optimizer\n        self.loss_fn = loss_fn\n        self.num_actions = num_actions\n        self.epsilon = epsilon\n        self.epsilon_decay = epsilon_decay\n        self.epsilon_min = epsilon_min\n        self.gamma = gamma\n        self.memory = []\n        self.batch_size = 32\n        self.max_memory_size = 1000\n    \n    def remember(self, state, action, reward, next_state, done):\n        if len(self.memory) > self.max_memory_size:\n            self.memory.pop(0)\n        self.memory.append((state, action, reward, next_state, done))\n    \n    def act(self, state):\n        if np.random.rand() <= self.epsilon:\n            return np.random.choice(self.num_actions)\n        state = torch.FloatTensor(state).unsqueeze(0)\n        q_values = self.model(state)\n        return torch.argm

In [10]:
"""
import gym
import gym_quantcircuit
import numpy as np
import matplotlib as plt

env = gym.make('quantcircuit-v0')
agent = DQNAgent(model, optimizer, loss_fn, num_actions)

episodes = 100
num_qubits = 4
test_goal_state = [0j] * (2**num_qubits - 1) + [1+0j]
env.var_init(num_features,
             unitary=False,
             gate_group='pauli',
             connectivity='fully_connected',
             X_train=X_train,
             Y_train=y_train,
             X_test=X_test,
             Y_test=y_test,
             feature_map=feature_map,
             goal_state=test_goal_state)

for e in range(episodes):
    print(f"Episode {e+1}/{episodes}")
    state = env.reset()
    done = False
    total_reward = 0
    
    while not done:
        
        action = env.sample()
        env.gate_list[action]
        print(env.gate_list[action])
        next_state, reward, done, _ = env.step(action)
        
        
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        total_reward += reward
        
        if done:
            print(f"Episode {e+1}/{episodes} - Reward: {total_reward}")
            break
    
    agent.replay()
"""


'\nimport gym\nimport gym_quantcircuit\nimport numpy as np\nimport matplotlib as plt\n\nenv = gym.make(\'quantcircuit-v0\')\nagent = DQNAgent(model, optimizer, loss_fn, num_actions)\n\nepisodes = 100\nnum_qubits = 4\ntest_goal_state = [0j] * (2**num_qubits - 1) + [1+0j]\nenv.var_init(num_features,\n             unitary=False,\n             gate_group=\'pauli\',\n             connectivity=\'fully_connected\',\n             X_train=X_train,\n             Y_train=y_train,\n             X_test=X_test,\n             Y_test=y_test,\n             feature_map=feature_map,\n             goal_state=test_goal_state)\n\nfor e in range(episodes):\n    print(f"Episode {e+1}/{episodes}")\n    state = env.reset()\n    done = False\n    total_reward = 0\n    \n    while not done:\n        \n        action = env.sample()\n        env.gate_list[action]\n        print(env.gate_list[action])\n        next_state, reward, done, _ = env.step(action)\n        \n        \n        agent.remember(state, action, r

In [9]:
import gym
import gym_quantcircuit
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

env = gym.make('quantcircuit-v0')

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if GPU is to be used
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)


In [10]:

Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)



In [11]:
class DQN(nn.Module):

    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_actions)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)
    

In [12]:
# BATCH_SIZE is the number of transitions sampled from the replay buffer
# GAMMA is the discount factor as mentioned in the previous section
# EPS_START is the starting value of epsilon
# EPS_END is the final value of epsilon
# EPS_DECAY controls the rate of exponential decay of epsilon, higher means a slower decay
# TAU is the update rate of the target network
# LR is the learning rate of the ``AdamW`` optimizer
BATCH_SIZE = 5
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 1000
TAU = 0.005
LR = 1e-4
test_goal_state = [0j] * (2**num_features - 1) + [1+0j]
env.var_init(num_qubits=num_features,
             unitary=True,
             gate_group='clifford',
             connectivity='fully_connected',
             X_train=X_train,
             Y_train=y_train,
             X_test=X_test,
             Y_test=y_test,
             feature_map=feature_map,
             goal_state=test_goal_state)

# Get number of actions from gym action space
n_actions = env.get_num_actions()

# Get the number of state observations
state, info = env.reset()
n_observations = 1

policy_net = DQN(n_observations, n_actions).to(device)
target_net = DQN(n_observations, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(10000)


steps_done = 0


def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1

    # Assicurati che lo stato sia 2D: (1, num_features)
    if len(state.shape) == 1:
        state = state.unsqueeze(0)
    elif len(state.shape) != 2:
        raise ValueError(f"Forma dello stato inaspettata: {state.shape}")

    if sample > eps_threshold:
        with torch.no_grad():
            # Stampa la forma dello stato per il debug
            print("Forma dello stato prima di passare alla policy_net:", state.shape)
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[env.sample()]], device=device, dtype=torch.long)






episode_durations = []


def plot_durations(show_result=False):
    plt.figure(1)
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())

gate_list:  [((0, 1), <bound method QuantumCircuit.cx of <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x151e96f50>>), ((0, 2), <bound method QuantumCircuit.cx of <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x151e96f50>>), ((0, 3), <bound method QuantumCircuit.cx of <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x151e96f50>>), ((1, 0), <bound method QuantumCircuit.cx of <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x151e96f50>>), ((1, 2), <bound method QuantumCircuit.cx of <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x151e96f50>>), ((1, 3), <bound method QuantumCircuit.cx of <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x151e96f50>>), ((2, 0), <bound method QuantumCircuit.cx of <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x151e96f50>>), ((2, 1), <bound method QuantumCircuit.cx of <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x151e96f50>>), ((2, 3), <bound method QuantumCircuit.cx of <qiskit.circuit

In [13]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    print("Ottimizzazione del modello")
    
    # Extract a batch of transitions from the replay buffer
    transitions = memory.sample(BATCH_SIZE)
    
    # Convert the batch of transitions into a batch of arrays (from Transition of batch)
    batch = Transition(*zip(*transitions))

    # Concatenate the elements of the batch and check dimensions
    state_batch = torch.cat(batch.state).view(BATCH_SIZE, -1)

    # Ensure actions are scalars and convert them to 1D tensors
    action_batch = [a.view(-1) if a.dim() == 0 else a for a in batch.action]
    action_batch = torch.stack(action_batch).view(BATCH_SIZE, -1)
    
    reward_batch = torch.cat(batch.reward)
    
    # Calculate Q(s_t, a) using the policy network
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Calculate V(s_{t+1}) for all next states
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None, batch.next_state)), 
                                  device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])
    
    # Ensure non_final_next_states has the correct shape
    if non_final_next_states.dim() == 1:
        non_final_next_states = non_final_next_states.unsqueeze(1)
    
    # Initialize next state values with zeros
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    
    # Calculate V(s_{t+1}) for non-terminal states
    with torch.no_grad():
        if len(non_final_next_states) > 0:
            next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values

    # Compute expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Calculate Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()

    # Clip gradients to prevent exploding gradients
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    
    # Optimizer step
    optimizer.step()

    print("Modello ottimizzato con successo!")

In [14]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    print("Ottimizzazione del modello")

    transitions = memory.sample(BATCH_SIZE)
    batch = Transition(*zip(*transitions))

    # Diagnostica le forme degli stati
    for i, state in enumerate(batch.state):
        print(f"State {i} shape: {state.shape} (dim: {state.dim()})")

    # Uniforma le dimensioni degli stati
    try:
        state_batch = torch.cat([s.unsqueeze(0) if s.dim() == 1 else s for s in batch.state])
        print(f"state_batch shape: {state_batch.shape}")
    except Exception as e:
        print(f"Errore durante la concatenazione degli stati: {e}")
        return

    # Assicurati che l'azione sia un tensore di tipo long
    try:
        action_batch = torch.cat([a.view(-1) if a.dim() == 0 else a for a in batch.action]).view(BATCH_SIZE, -1)
        print(f"action_batch shape: {action_batch.shape}")
    except Exception as e:
        print(f"Errore durante la concatenazione delle azioni: {e}")
        return

    # Assicurati che la ricompensa sia un tensore float
    try:
        reward_batch = torch.cat(batch.reward).float()
        print(f"reward_batch shape: {reward_batch.shape}")
    except Exception as e:
        print(f"Errore durante la concatenazione delle ricompense: {e}")
        return

    # Calcola Q(s_t, a) usando la rete policy
    state_action_values = policy_net(state_batch).gather(1, action_batch)
    print(f"state_action_values shape: {state_action_values.shape}")

    # Crea una maschera per gli stati non finali
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None, batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])

    # Assicurati che non_final_next_states abbia la forma corretta
    if non_final_next_states.dim() == 1:
        non_final_next_states = non_final_next_states.unsqueeze(1)

    # Inizializza i valori degli stati successivi con zero
    next_state_values = torch.zeros(BATCH_SIZE, device=device)

    with torch.no_grad():
        if len(non_final_next_states) > 0:
            # Calcola V(s_{t+1}) per stati non terminali
            next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0]

    # Calcola i valori attesi di Q
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Calcola la perdita usando Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Ottimizza il modello
    optimizer.zero_grad()
    loss.backward()

    # Clippa i gradienti per prevenire l'esplosione dei gradienti
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)

    # Esegui un passo di ottimizzazione
    optimizer.step()

    print(f"Perdita: {loss.item()}")
    print("Modello ottimizzato con successo!")



In [17]:
"""if torch.cuda.is_available() or torch.backends.mps.is_available():
    num_episodes = 50
else:
    num_episodes = 50
episode_results=[]
for i_episode in range(num_episodes):
    # Initialize the environment and get its state
    print("Episiodio numero: "+ str(i_episode+1)+"/"+str(num_episodes))
    state, info = env.reset()
    #print(info.get('current_accuracy'))
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    for t in count():
        print(state.shape)
        action = select_action(state)
        observation, reward, terminated, _ = env.step(action)
        print("La reward sul test set è :"+str(reward))
        print("L' episiodio risulta essere: "+str(terminated))
        
        #reward = torch.tensor([reward], device=device)
        reward = torch.tensor([reward], dtype=torch.float32, device=device)

        done = terminated

        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)
        print("Lo stato :"+str(state))
        print("L'action è :"+str(action))
        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)
        print("Pesi aggiornati")
        if done:
            episode_durations.append(t + 1)
            #plot_durations()
            break
        # Store the accuracy and rewards for this episode
        episode_results.append({
            "episode": i_episode,
            "final_reward": reward.item(),  # Convert to a regular float
            "duration": t + 1,
            "accuracy": info.get('current_accuracy')
        })

# Print all results after the loop
print(info.get('circuit'))
for result in episode_results:
    print(f"Episode {result['episode']}: Reward: {result['final_reward']}, Duration: {result['duration']}, Accuracy: {result['accuracy']}")


print('Complete')
#plot_durations(show_result=True)
#plt.ioff()
#plt.show()
"""


'if torch.cuda.is_available() or torch.backends.mps.is_available():\n    num_episodes = 50\nelse:\n    num_episodes = 50\nepisode_results=[]\nfor i_episode in range(num_episodes):\n    # Initialize the environment and get its state\n    print("Episiodio numero: "+ str(i_episode+1)+"/"+str(num_episodes))\n    state, info = env.reset()\n    #print(info.get(\'current_accuracy\'))\n    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)\n    for t in count():\n        print(state.shape)\n        action = select_action(state)\n        observation, reward, terminated, _ = env.step(action)\n        print("La reward sul test set è :"+str(reward))\n        print("L\' episiodio risulta essere: "+str(terminated))\n        \n        #reward = torch.tensor([reward], device=device)\n        reward = torch.tensor([reward], dtype=torch.float32, device=device)\n\n        done = terminated\n\n        if terminated:\n            next_state = None\n        else:\n            next_

In [18]:
"""import torch
from itertools import count

if torch.cuda.is_available() or torch.backends.mps.is_available():
    device = torch.device("cuda" if torch.cuda.is_available() else "mps")
else:
    device = torch.device("cpu")

num_episodes = 50
episode_results = []

for i_episode in range(num_episodes):
    # Initialize the environment and get its state
    print("Episodio numero: " + str(i_episode + 1) + "/" + str(num_episodes))
    state, info = env.reset()
    print("lo stato prima del tensore è: " + str(state))
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    
    for t in count():
        print(state.shape)
        action = select_action(state)
        observation, reward, terminated, _ = env.step(action)
        print("La reward sul test set è: " + str(reward))
        print("L'episodio risulta essere: " + str(terminated))
        
        reward = torch.tensor([reward], dtype=torch.float32, device=device)
        done = terminated

        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)
        
        print("Lo stato: " + str(state))
        print("L'action è: " + str(action))
        
        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 − τ)θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key] * TAU + target_net_state_dict[key] * (1 - TAU)
        target_net.load_state_dict(target_net_state_dict)
        print("Pesi aggiornati")
        
        if done:
            episode_durations.append(t + 1)
            break
        
        # Store the accuracy and rewards for this episode
        episode_results.append({
            "episode": i_episode,
            "final_reward": reward.item(),  # Convert to a regular float
            "duration": t + 1,
            "accuracy": info.get('current_accuracy')
        })

# Print all results after the loop
print(info.get('circuit'))
for result in episode_results:
    print(f"Episode {result['episode']}: Reward: {result['final_reward']}, Duration: {result['duration']}, Accuracy: {result['accuracy']}")

print('Complete')
"""
device = torch.device("mps" if torch.backends.mps.is_built() else "cpu")
num_episodes = 15
episode_results = []

for i_episode in range(num_episodes):
    print(f"Episodio numero: {i_episode+1}/{num_episodes}")
    state, info = env.reset()
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)  # Aggiungi dimensione batch

    for t in count():
        print(f"Forma dello stato: {state.shape}")
        if state.dim() == 1:  # Se è un tensore 1D, espandilo a 2D
            state = state.unsqueeze(0)  # Cambia la forma a (1, 813)
        action = select_action(state)
        observation, reward, terminated, measures = env.step(action)

        print(f"La reward sul test set è: {reward}")
        print(f"L'episodio risulta essere: {terminated}")

        # Converti la reward in un tensore float
        reward = torch.tensor([reward], dtype=torch.float32, device=device)

        done = terminated

        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

        print(f"Lo stato: {state}")
        print(f"L'azione è: {action}")

        # Memorizza la transizione nella memoria di replay
        memory.push(state, action, next_state, reward)

        # Passa allo stato successivo
        state = next_state

        # Ottimizza il modello
        optimize_model()

        # Soft update della rete target
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key] * TAU + target_net_state_dict[key] * (1 - TAU)
        target_net.load_state_dict(target_net_state_dict)
        print("Pesi aggiornati")

        if done:
            episode_durations.append(t + 1)
            #plot_durations()
            break

        # Memorizza i risultati dell'episodio
        episode_results.append({
            "episode": i_episode,
            "final_reward": reward.item(),
            "duration": t + 1,
            "accuracy": measures.get('current_accuracy', 0.0)
        })

# Stampa tutti i risultati dopo il ciclo
print("Circuito finale:")
print(info.get('circuit'))
for result in episode_results:
    print(f"Episodio {result['episode']}: Reward: {result['final_reward']}, Durata: {result['duration']}, Accuracy: {result['accuracy']}")

print('Addestramento Completo')


Episodio numero: 1/15
Forma dello stato: torch.Size([1])
l'azione schelta è:9
Current Quantum Circuit:
     
q_0: 
     
q_1: 
     
q_2: 
     
q_3: 
     
[0 1 0 0 0 0 2 1 1 2 1 1 2 1 0 2 1 0 2 2 1 2 2 2 1 2 0 1 0 1 0 1 2 1 1 2 1
 1 1 0 2 0 0 0 0 1 1 0 2 0 1 1 2 0]
Quantum Circuit:
     ┌───┐
q_0: ┤ X ├
     └─┬─┘
q_1: ──┼──
       │  
q_2: ──┼──
       │  
q_3: ──■──
          
FEATURE MAP:
      ┌──────────────┐┌────────────────────────────────────┐
q2_0: ┤0             ├┤0                                   ├
      │              ││                                    │
q2_1: ┤1             ├┤1                                   ├
      │  circuit-202 ││  ZZFeatureMap(x[0],x[1],x[2],x[3]) │
q2_2: ┤2             ├┤2                                   ├
      │              ││                                    │
q2_3: ┤3             ├┤3                                   ├
      └──────────────┘└────────────────────────────────────┘
Cross-validation scores: [0.8  0.8  0.72 0.76 0.75]
Av

In [19]:
# Salva il modello addestrato
torch.save({
    'policy_net_state_dict': policy_net.state_dict(),
    'target_net_state_dict': target_net.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}, 'rl-wine.pth')
print("Modello salvato con successo!")


Modello salvato con successo!


In [16]:
# Ricarica il modello salvato
checkpoint = torch.load('rl-wine.pth')
policy_net.load_state_dict(checkpoint['policy_net_state_dict'])
target_net.load_state_dict(checkpoint['target_net_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
print("Modello ricaricato con successo!")
print(policy_net)



Modello ricaricato con successo!
DQN(
  (layer1): Linear(in_features=1, out_features=128, bias=True)
  (layer2): Linear(in_features=128, out_features=128, bias=True)
  (layer3): Linear(in_features=128, out_features=28, bias=True)
)


/var/folders/7f/7v0y2r7x71d8fvv9hcm5rtx80000gn/T/ipykernel_48739/3685394206.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('rl-wine.pth')


In [17]:
def generate_quantum_circuit(agent, env):
    state, info = env.reset()
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    done = False
    while not done:
        action = select_action(state)
        state, reward, done, _ = env.step(action)
        state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)

# Genera il circuito quantistico
test_goal_state = [0j] * (2**num_features - 1) + [1+0j]
env.var_init(num_qubits=num_features,
             unitary=False,
             gate_group='pauli',
             connectivity='fully_connected',
             X_train=X_train,
             Y_train=y_train,
             X_test=X_test,
             Y_test=y_test,
             feature_map=feature_map,
             goal_state=test_goal_state)

quantum_circuit = generate_quantum_circuit(policy_net, env)
print("Circuito quantistico generato con successo!")


gate_list:  [((0,), <bound method QuantumCircuit.id of <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x155b32a40>>), ((1,), <bound method QuantumCircuit.id of <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x155b32a40>>), ((2,), <bound method QuantumCircuit.id of <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x155b32a40>>), ((3,), <bound method QuantumCircuit.id of <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x155b32a40>>), ((0,), <bound method QuantumCircuit.h of <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x155b32a40>>), ((1,), <bound method QuantumCircuit.h of <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x155b32a40>>), ((2,), <bound method QuantumCircuit.h of <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x155b32a40>>), ((3,), <bound method QuantumCircuit.h of <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x155b32a40>>), ((0,), <bound method QuantumCircuit.x of <qiskit.circuit.quantumcircuit.Quantum